# Install the Libraries

In [3]:
!pip install tensorflow_decision_forests kaggle

# Grab some data

In [4]:
%%bash 
mkdir ~/.kaggle
cp kaggle.json  ~/.kaggle/kaggle.json
chmod 600 ~/.kaggle/kaggle.json
kaggle datasets download -d ahsan81/used-handheld-device-data
unzip used-handheld-device-data.zip

used-handheld-device-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  used-handheld-device-data.zip


replace used_device_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


CalledProcessError: Command 'b'mkdir ~/.kaggle\ncp kaggle.json  ~/.kaggle/kaggle.json\nchmod 600 ~/.kaggle/kaggle.json\nkaggle datasets download -d ahsan81/used-handheld-device-data\nunzip used-handheld-device-data.zip\n'' returned non-zero exit status 1.

# Import Dependencies

In [7]:
import tensorflow_decision_forests as tfdf
import pandas as pd
from sklearn.model_selection import train_test_split

2023-01-27 02:13:59.519794: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 02:14:00.667604: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-27 02:14:00.667726: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-27 02:14:00.667737: W tensorfl

# Create Training and Testing Partitions

In [8]:
df = pd.read_csv('used_device_data.csv')
train, test = train_test_split(df, test_size=.2, random_state=1234) 

In [9]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="normalized_used_price", task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="normalized_used_price", task=tfdf.keras.Task.REGRESSION)

# Fit the models 

In [10]:
model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
model.fit(train_ds)

Use /tmp/tmpmvs1b412 as temporary training directory
Reading training dataset...
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Training dataset read in 0:00:03.896372. Found 2763 examples.
Training model...


[INFO 2023-01-27T02:14:33.397107058+00:00 kernel.cc:1214] Loading model from path /tmp/tmpmvs1b412/model/ with prefix 3155fb5a01b74af2


Model trained in 0:00:01.745957
Compiling model...


[INFO 2023-01-27T02:14:34.159620628+00:00 decision_forest.cc:661] Model loaded with 300 root(s), 256618 node(s), and 14 input feature(s).
[INFO 2023-01-27T02:14:34.159664074+00:00 abstract_model.cc:1311] Engine "RandomForestGeneric" built
[INFO 2023-01-27T02:14:34.159693122+00:00 kernel.cc:1046] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


# Evaluate

In [11]:
model.compile(metrics=["mse"])
evaluation = model.evaluate(test_ds, return_dict=True)

1/1 [==============================] - 0s 384ms/step - loss: 0.0000e+00 - mse: 0.0557


# View one Tree

In [14]:
%matplotlib inline 
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0, max_depth=3)